# Scratch Work

Move along - nothing to see here.

In [1]:
import algebras as alg
import json
import os

In [2]:
import itertools as it

In [3]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

In [4]:
s3_path = os.path.join(alg_dir, "s3_symmetric_group_on_3_letters.json")
!cat {s3_path}

{"type": "Group",
 "name": "S3",
 "description": "Symmetric group on 3 letters",
 "element_names": ["e", "r", "r^2", "f", "fr", "rf"],
 "mult_table": [[0, 1, 2, 3, 4, 5],
                [1, 2, 0, 5, 3, 4],
                [2, 0, 1, 4, 5, 3],
                [3, 4, 5, 0, 1, 2],
                [4, 5, 3, 2, 0, 1],
                [5, 3, 4, 1, 2, 0]]
}


In [5]:
s3 = alg.Group(s3_path)
s3.print_info()


Group : S3 : Symmetric group on 3 letters
  Element Names: ['e', 'r', 'r^2', 'f', 'fr', 'rf']
  Is Abelian? False
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(r) = r^2 
    inv(r^2) = r 
    inv(f) = f   **
    inv(fr) = fr   **
    inv(rf) = rf   **
  Is associative? True
  Cayley Table:
      e    r  r^2    f   fr   rf
      r  r^2    e   rf    f   fr
    r^2    e    r   fr   rf    f
      f   fr   rf    e    r  r^2
     fr   rf    f  r^2    e    r
     rf    f   fr    r  r^2    e


In [6]:
element_subset = s3.element_names[:2]
element_subset

['e', 'r']

In [7]:
s3.closure(element_subset)

['e', 'r^2', 'r']

In [8]:
z2 = alg.generate_cyclic_group(2)
z2xz2 = z2 * z2
z2xz2xz2 = z2 * z2 * z2
z2.print_info()
z2xz2.print_info()
z2xz2xz2.print_info()


Group : Z2 : Cyclic group of order 2
  Element Names: ['e', 'a']
  Is Abelian? True
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(a) = a   **
  Is associative? True
  Cayley Table:
    e  a
    a  e

Group : Z2_x_Z2 : Direct product of Z2 & Z2
  Element Names: ['e:e', 'e:a', 'a:e', 'a:a']
  Is Abelian? True
  Inverses:  (** - indicates that it is its own inverse)
    inv(e:e) = e:e   **
    inv(e:a) = e:a   **
    inv(a:e) = a:e   **
    inv(a:a) = a:a   **
  Is associative? True
  Cayley Table:
    e:e  e:a  a:e  a:a
    e:a  e:e  a:a  a:e
    a:e  a:a  e:e  e:a
    a:a  a:e  e:a  e:e

Group : Z2_x_Z2_x_Z2 : Direct product of Z2_x_Z2 & Z2
  Element Names: ['e:e:e', 'e:e:a', 'e:a:e', 'e:a:a', 'a:e:e', 'a:e:a', 'a:a:e', 'a:a:a']
  Is Abelian? True
  Inverses:  (** - indicates that it is its own inverse)
    inv(e:e:e) = e:e:e   **
    inv(e:e:a) = e:e:a   **
    inv(e:a:e) = e:a:e   **
    inv(e:a:a) = e:a:a   **
    inv(a:e:e) = a:e:e   **
    in

In [9]:
z2xz2xz2.closed_subsets_of_elements()

[['a:e:e', 'e:e:e', 'a:e:a', 'e:e:a'],
 ['a:e:e', 'e:e:e', 'e:a:a', 'a:a:a'],
 ['e:e:e', 'e:a:e', 'a:e:a', 'a:a:a'],
 ['e:e:e', 'a:a:a'],
 ['e:e:e', 'e:a:e'],
 ['e:e:e', 'e:a:a'],
 ['a:e:e', 'e:e:e'],
 ['a:e:e', 'e:e:e', 'e:a:e', 'a:a:e'],
 ['e:e:e', 'a:a:e'],
 ['e:e:a', 'a:a:e', 'a:a:a', 'e:e:e'],
 ['e:e:e', 'e:a:a', 'a:e:a', 'a:a:e'],
 ['e:e:e', 'a:e:a'],
 ['e:e:e', 'e:a:e', 'e:a:a', 'e:e:a'],
 ['e:e:e', 'e:e:a']]

In [10]:
len(z2xz2xz2.closed_subsets_of_elements())

14

In [11]:
s3.element_names

['e', 'r', 'r^2', 'f', 'fr', 'rf']

In [12]:
s3.closed_subsets_of_elements()

[['fr', 'e'], ['rf', 'e'], ['e', 'r^2', 'r'], ['e', 'f']]

In [13]:
foo = ['r', 'r^2', 'e']

In [14]:
sorted(foo, key=lambda x: s3.element_names.index(x))

['e', 'r', 'r^2']

## Dihedral Group of Order 6

[See Wikipedia](https://en.wikipedia.org/wiki/Dihedral_group_of_order_6)

In [15]:
d3_path = os.path.join(alg_dir, "d3_dihedral_group_of_order_6.json")

!cat {d3_path}

{"type": "Group",
 "name": "D3",
 "description": "https://en.wikipedia.org/wiki/Dihedral_group_of_order_6",
 "element_names": ["e", "a", "b", "aba", "ab", "ba"],
 "mult_table": [[0, 1, 2, 3, 4, 5],
                [1, 0, 4, 5, 2, 3],
                [2, 5, 0, 4, 3, 1],
                [3, 4, 5, 0, 1, 2],
                [4, 3, 1, 2, 5, 0],
                [5, 2, 3, 1, 0, 4]]
}

In [16]:
d3 = alg.Group(d3_path)

d3.print_info()


Group : D3 : https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
  Element Names: ['e', 'a', 'b', 'aba', 'ab', 'ba']
  Is Abelian? False
  Inverses:  (** - indicates that it is its own inverse)
    inv(e) = e   **
    inv(a) = a   **
    inv(b) = b   **
    inv(aba) = aba   **
    inv(ab) = ba 
    inv(ba) = ab 
  Is associative? True
  Cayley Table:
      e    a    b  aba   ab   ba
      a    e   ab   ba    b  aba
      b   ba    e   ab  aba    a
    aba   ab   ba    e    a    b
     ab  aba    a    b   ba    e
     ba    b  aba    a    e   ab


In [17]:
d3.closed_subsets_of_elements()

[['e', 'aba'], ['ab', 'ba', 'e'], ['e', 'a'], ['e', 'b']]

In [18]:
d3.element_names

['e', 'a', 'b', 'aba', 'ab', 'ba']

In [19]:
d3.mult_table

array([[0, 1, 2, 3, 4, 5],
       [1, 0, 4, 5, 2, 3],
       [2, 5, 0, 4, 3, 1],
       [3, 4, 5, 0, 1, 2],
       [4, 3, 1, 2, 5, 0],
       [5, 2, 3, 1, 0, 4]])

In [20]:
sorted(['ba', 'ab', 'e'], key=lambda x: d3.element_names.index(x))

['e', 'ab', 'ba']

In [21]:
d3.closed_subsets_of_elements()

[['e', 'aba'], ['ab', 'ba', 'e'], ['e', 'a'], ['e', 'b']]

In [22]:
d3.proper_subgroups()

[Group('D3_subgroup_0', 'Subgroup of: https://en.wikipedia.org/wiki/Dihedral_group_of_order_6', ['e', 'aba'], [[0 1]
  [1 0]]) ,
 Group('D3_subgroup_1', 'Subgroup of: https://en.wikipedia.org/wiki/Dihedral_group_of_order_6', ['e', 'ab', 'ba'], [[0 1 2]
  [1 2 0]
  [2 0 1]]) ,
 Group('D3_subgroup_2', 'Subgroup of: https://en.wikipedia.org/wiki/Dihedral_group_of_order_6', ['e', 'a'], [[0 1]
  [1 0]]) ,
 Group('D3_subgroup_3', 'Subgroup of: https://en.wikipedia.org/wiki/Dihedral_group_of_order_6', ['e', 'b'], [[0 1]
  [1 0]]) ]

In [23]:
z2xz2xz2.subgroups()

[Group('Z2_x_Z2_x_Z2_subgroup_0', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'e:e:a', 'a:e:e', 'a:e:a'], [[0 1 2 3]
  [1 0 3 2]
  [2 3 0 1]
  [3 2 1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_1', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'e:a:a', 'a:e:e', 'a:a:a'], [[0 1 2 3]
  [1 0 3 2]
  [2 3 0 1]
  [3 2 1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_2', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'e:a:e', 'a:e:a', 'a:a:a'], [[0 1 2 3]
  [1 0 3 2]
  [2 3 0 1]
  [3 2 1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_3', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'a:a:a'], [[0 1]
  [1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_4', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'e:a:e'], [[0 1]
  [1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_5', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'e:a:a'], [[0 1]
  [1 0]]) ,
 Group('Z2_x_Z2_x_Z2_subgroup_6', 'Subgroup of: Direct product of Z2_x_Z2 & Z2', ['e:e:e', 'a:e:e'], [[0 1]
  [1 0]]) ,
 Group('Z2